<a href="https://colab.research.google.com/github/Huangjian2013/ai-demo/blob/main/rag/10-LangChain-pdf-rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain --quiet
!pip install langchain-openai --quiet

In [ ]:
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.output_parsers import StrOutputParser



In [ ]:
# 使用的是Cloudflare的OpenAPI封装接口
model = ChatOpenAI(model="@cf/meta/llama-2-7b-chat-int8",
          api_key = userdata.get('OPEN_AI_KEY'),
          base_url = userdata.get('OPEN_AI_BASE_URL'))

In [ ]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "将以下输入翻译成英文"),
    ("human", "{input}")
])

parser = StrOutputParser()

chain = prompt_template | model | parser

chain.invoke("我想买车了")

'I want to buy a car'

In [ ]:
chain = prompt_template | model | parser

back_to_Chinese_Prompt = ChatPromptTemplate.from_messages([
    ("system", "将以下输入翻译成中文。按以下格式输出：二次翻译 = <result>。比如输入'good'，输出'二次翻译 = 好'"),
    ("human", "{input}")
])
composed_chain = {"input": chain} | back_to_Chinese_Prompt | model | parser

composed_chain.invoke("我想买车了")

'二次翻译 = 我要购买一辆汽车。'

In [ ]:
composed_chain = chain | (lambda x: {"input": x}) | back_to_Chinese_Prompt | model | parser

composed_chain.invoke("买什么车好")


'二次翻译 = 这样的汽车购买有什么好选择？'

In [ ]:
from langchain_core.runnables import RunnableParallel

chain_with_pipe = (RunnableParallel({"input": chain})
                    .pipe(back_to_Chinese_Prompt)
                    .pipe(model)
                    .pipe(parser)
)

chain_with_pipe.invoke("买ROBO-01吧")

'二次翻译 = 购买ROBO-01！'

In [ ]:
first_chain = {"input": chain} | back_to_Chinese_Prompt | model | parser
sec_chain = chain
map_chain = RunnableParallel(chinese=first_chain, english=sec_chain)
map_chain.invoke("买买买！快买吧!")


{'chinese': '二次翻译 =.NAME好 buys!".', 'english': 'Buy buy buy! Buy quickly!'}

In [ ]:
chain = prompt_template | model | parser

async for chunk in chain.astream("异步买车的过程"):
    print(chunk, end="|", flush=True)

chain_with_pipe.invoke("买Robo-01吧")

Here| is| the| translation| of| "|异|步|买|车|的|过程|"| to| English|:

|"As|ynchronous| Car| Buying| Process|"|

'二次翻译 = 让我们购买Robo-01。'